In [1]:
import json
import pandas as pd
import numpy as np

import re

from sqlalchemy import create_engine
import psycopg2

from config import db_password

import time

In [2]:
# 1. Add the clean movie function that takes in the argument, "movie".
def clean_movie(movie):
    movie = dict(movie) #create a non-destructive copy
    alt_titles = {}
    # combine alternate titles into one list
    for key in ['Also known as','Arabic','Cantonese','Chinese','French',
                'Hangul','Hebrew','Hepburn','Japanese','Literally',
                'Mandarin','McCune-Reischauer','Original title','Polish',
                'Revised Romanization','Romanized','Russian',
                'Simplified','Traditional','Yiddish']:
        if key in movie:
            alt_titles[key] = movie[key]
            movie.pop(key)
    if len(alt_titles) > 0:
        movie['alt_titles'] = alt_titles

    # merge column names
    def change_column_name(old_name, new_name):
        if old_name in movie:
            movie[new_name] = movie.pop(old_name)
    change_column_name('Adaptation by', 'Writer(s)')
    change_column_name('Country of origin', 'Country')
    change_column_name('Directed by', 'Director')
    change_column_name('Distributed by', 'Distributor')
    change_column_name('Edited by', 'Editor(s)')
    change_column_name('Length', 'Running time')
    change_column_name('Original release', 'Release date')
    change_column_name('Music by', 'Composer(s)')
    change_column_name('Produced by', 'Producer(s)')
    change_column_name('Producer', 'Producer(s)')
    change_column_name('Productioncompanies ', 'Production company(s)')
    change_column_name('Productioncompany ', 'Production company(s)')
    change_column_name('Released', 'Release Date')
    change_column_name('Release Date', 'Release date')
    change_column_name('Screen story by', 'Writer(s)')
    change_column_name('Screenplay by', 'Writer(s)')
    change_column_name('Story by', 'Writer(s)')
    change_column_name('Theme music composer', 'Composer(s)')
    change_column_name('Written by', 'Writer(s)')
   

    return movie

In [3]:
# 2 Add the function that takes in three arguments;
# Wikipedia data, Kaggle metadata, and MovieLens rating data (from Kaggle)

def extract_transform_load():
    # Read in the kaggle metadata and MovieLens ratings CSV files as Pandas DataFrames.
    kaggle_metadata = pd.read_csv(kaggle_file, low_memory=False)
    ratings = pd.read_csv(ratings_file)

    # Open and read the Wikipedia data JSON file.
    with open(wiki_file, mode='r') as file:
        wiki_movies_raw = json.load(file)
    
    # 3. Write a list comprehension to filter out TV shows.
    wiki_movies = [movie for movie in wiki_movies_raw
               if ('Director' in movie or 'Directed by' in movie)
                   and 'imdb_link' in movie
                   and 'No. of episodes' not in movie]

    # 4. Write a list comprehension to iterate through the cleaned wiki movies list
    # and call the clean_movie function on each movie.
    clean_movies = [clean_movie(movie) for movie in wiki_movies]

    # 5. Read in the cleaned movies list from Step 4 as a DataFrame.
    wiki_movies_df = pd.DataFrame(clean_movies)

    # 6. Write a try-except block to catch errors while extracting the IMDb ID using a regular expression string and
    #  dropping any imdb_id duplicates. If there is an error, capture and print the exception.
    try:
        wiki_movies_df['imdb_id'] = wiki_movies_df['imdb_link'].str.extract(r'(tt\d{7})')
        wiki_movies_df.drop_duplicates(subset='imdb_id', inplace=True)
    except Exception as e:
        print(e)
        
    #  7. Write a list comprehension to keep the columns that don't have null values from the wiki_movies_df DataFrame.
    wiki_columns_to_keep = [column for column in wiki_movies_df.columns if wiki_movies_df[column].isnull().sum() < len(wiki_movies_df) * 0.9]
    wiki_movies_df = wiki_movies_df[wiki_columns_to_keep]

    # 8. Create a variable that will hold the non-null values from the “Box office” column.
    box_office = wiki_movies_df['Box office'].dropna()
    
    # 9. Convert the box office data created in Step 8 to string values using the lambda and join functions.
    box_office[box_office.map(lambda x: type(x) != str)]
    box_office = box_office.apply(lambda x: ' '.join(x) if type(x) == list else x)

    # 10. Write a regular expression to match the six elements of "form_one" of the box office data.
    form_one = r'\$\s*\d+\.?\d*\s*[mb]illion'
    matches_form_one = box_office.str.contains(form_one, flags=re.IGNORECASE, na=False)

    # 11. Write a regular expression to match the three elements of "form_two" of the box office data.
    form_two = r'\$\s*\d{1,3}(?:[,\.]\d{3})+(?!\s[mb]illion)'
    matches_form_two = box_office.str.contains(form_two, flags=re.IGNORECASE, na=False)

    # 12. Add the parse_dollars function.
    def parse_dollars(s):
    # if s is not a string, return NaN
        if type(s) != str:
            return np.nan

    # if input is of the form $###.# million
        if re.match(r'\$\s*\d+\.?\d*\s*milli?on', s, flags=re.IGNORECASE):

        # remove dollar sign and " million"
            s = re.sub('\$|\s|[a-zA-Z]','', s)

        # convert to float and multiply by a million
            value = float(s) * 10**6

        # return value
            return value

    # if input is of the form $###.# billion
        elif re.match(r'\$\s*\d+\.?\d*\s*billi?on', s, flags=re.IGNORECASE):

        # remove dollar sign and " billion"
            s = re.sub('\$|\s|[a-zA-Z]','', s)

        # convert to float and multiply by a billion
            value = float(s) * 10**9

        # return value
            return value

    # if input is of the form $###,###,###
        elif re.match(r'\$\s*\d{1,3}(?:[,\.]\d{3})+(?!\s[mb]illion)', s, flags=re.IGNORECASE):

        # remove dollar sign and commas
            s = re.sub('\$|,','', s)

        # convert to float
            value = float(s)

        # return value
            return value

    # otherwise, return NaN
        else:
            return np.nan    

    
        
    # 13. Clean the box office column in the wiki_movies_df DataFrame.
    
    wiki_movies_df['box_office'] = box_office.str.extract(f'({form_one}|{form_two})', flags=re.IGNORECASE)[0].apply(parse_dollars)
    wiki_movies_df.drop('Box office', axis=1, inplace=True)
    
    # 14. Clean the budget column in the wiki_movies_df DataFrame.
    budget = wiki_movies_df['Budget'].dropna()
    budget = budget.map(lambda x: ' '.join(x) if type(x) == list else x)
    wiki_movies_df['budget'] = budget.str.extract(f'({form_one}|{form_two})', flags=re.IGNORECASE)[0].apply(parse_dollars)

    # 15. Clean the release date column in the wiki_movies_df DataFrame.
    date_form_one = r'(?:January|February|March|April|May|June|July|August|September|October|November|December)\s[123]?\d,\s\d{4}'
    date_form_two = r'\d{4}.[01]\d.[0123]\d'
    date_form_three = r'(?:January|February|March|April|May|June|July|August|September|October|November|December)\s\d{4}'
    date_form_four = r'\d{4}'
    
    release_date = wiki_movies_df['Release date'].dropna().apply(lambda x: ' '.join(x) if type(x) == list else x)
    release_date.str.extract(f'({date_form_one}|{date_form_two}|{date_form_three}|{date_form_four})', flags=re.IGNORECASE)
    wiki_movies_df['release_date'] = pd.to_datetime(release_date.str.extract(f'({date_form_one}|{date_form_two}|{date_form_three}|{date_form_four})')[0], infer_datetime_format=True)
    
    # 16. Clean the running time column in the wiki_movies_df DataFrame.
    running_time = wiki_movies_df['Running time'].dropna().apply(lambda x: ' '.join(x) if type(x) == list else x)
    running_time_extract = running_time.str.extract(r'(\d+)\s*ho?u?r?s?\s*(\d*)|(\d+)\s*m')
    wiki_movies_df['running_time'] = running_time_extract.apply(lambda row: row[0]*60 + row[1] if row[2] == 0 else row[2], axis=1)
    wiki_movies_df.drop('Running time', axis=1, inplace=True)
    # Return three variables. The first is the wiki_movies_df DataFrame
    
    return wiki_movies_df, kaggle_metadata, ratings 

In [4]:
# 17. Create the path to your file directory and variables for the three files.
file_dir = 'Resources'
# The Wikipedia data
wiki_file = f'{file_dir}/wikipedia-movies.json'
# The Kaggle metadata
kaggle_file = f'{file_dir}/movies_metadata.csv'
# The MovieLens rating data.
ratings_file = f'{file_dir}/ratings.csv'

In [5]:
# 18. Set the three variables equal to the function created in D1.
wiki_file, kaggle_file, ratings_file = extract_transform_load()

In [6]:
# 19. Set the wiki_movies_df equal to the wiki_file variable. 
wiki_movies_df = wiki_file

In [7]:
# 20. Check that the wiki_movies_df DataFrame looks like this. 
wiki_movies_df.head()

,url,year,imdb_link,title,Based on,Starring,Cinematography,Release date,Country,Language,...,Editor(s),Composer(s),Producer(s),Production company(s),Writer(s),imdb_id,box_office,budget,release_date,running_time
0,https://en.wikipedia.org/wiki/The_Adventures_o...,1990,https://www.imdb.com/title/tt0098987/,The Adventures of Ford Fairlane,"[Characters, by Rex Weiner]","[Andrew Dice Clay, Wayne Newton, Priscilla Pre...",Oliver Wood,"[July 11, 1990, (, 1990-07-11, )]",United States,English,...,Michael Tronick,"[Cliff Eidelman, Yello]","[Steve Perry, Joel Silver]",Silver Pictures,"[David Arnott, James Cappe]",tt0098987,21400000.0,20000000.0,1990-07-11,102
1,"https://en.wikipedia.org/wiki/After_Dark,_My_S...",1990,https://www.imdb.com/title/tt0098994/,"After Dark, My Sweet","[the novel, After Dark, My Sweet, by, Jim Thom...","[Jason Patric, Rachel Ward, Bruce Dern, George...",Mark Plummer,"[May 17, 1990, (, 1990-05-17, ), (Cannes Film ...",United States,English,...,Howard E. Smith,Maurice Jarre,"[Ric Kidney, Robert Redlin]",Avenue Pictures,"[James Foley, Robert Redlin]",tt0098994,2700000.0,6000000.0,1990-05-17,114
2,https://en.wikipedia.org/wiki/Air_America_(film),1990,https://www.imdb.com/title/tt0099005/,Air America,"[Air America, by, Christopher Robbins]","[Mel Gibson, Robert Downey Jr., Nancy Travis, ...",Roger Deakins,"[August 10, 1990, (, 1990-08-10, )]",United States,"[English, Lao]",...,"[John Bloom, Lois Freeman-Fox]",Charles Gross,Daniel Melnick,"[Carolco Pictures, IndieProd Company]","[John Eskow, Richard Rush]",tt0099005,57718089.0,35000000.0,1990-08-10,113
3,https://en.wikipedia.org/wiki/Alice_(1990_film),1990,https://www.imdb.com/title/tt0099012/,Alice,NaN,"[Alec Baldwin, Blythe Danner, Judy Davis, Mia ...",Carlo Di Palma,"[December 25, 1990, (, 1990-12-25, )]",United States,English,...,Susan E. Morse,NaN,Robert Greenhut,NaN,Woody Allen,tt0099012,7331647.0,12000000.0,1990-12-25,106
4,https://en.wikipedia.org/wiki/Almost_an_Angel,1990,https://www.imdb.com/title/tt0099018/,Almost an Angel,NaN,"[Paul Hogan, Elias Koteas, Linda Kozlowski]",Russell Boyd,"December 19, 1990",US,English,...,David Stiven,Maurice Jarre,John Cornell,NaN,Paul Hogan,tt0099018,6939946.0,25000000.0,1990-12-19,95


In [8]:
# 21. Check that wiki_movies_df DataFrame columns are correct. 
wiki_movies_df.columns.to_list()

['url',
 'year',
 'imdb_link',
 'title',
 'Based on',
 'Starring',
 'Cinematography',
 'Release date',
 'Country',
 'Language',
 'Budget',
 'Director',
 'Distributor',
 'Editor(s)',
 'Composer(s)',
 'Producer(s)',
 'Production company(s)',
 'Writer(s)',
 'imdb_id',
 'box_office',
 'budget',
 'release_date',
 'running_time']

In [9]:
wiki_movies_df.sample(10)

,url,year,imdb_link,title,Based on,Starring,Cinematography,Release date,Country,Language,...,Editor(s),Composer(s),Producer(s),Production company(s),Writer(s),imdb_id,box_office,budget,release_date,running_time
5957,https://en.wikipedia.org/wiki/Manhunt:_The_Sea...,2013,https://www.imdb.com/title/tt2475544/,Manhunt: The Search for Bin Laden,[Manhunt: The Ten-Year Search for Bin Laden Fr...,NaN,"[Frank-Peter Lehmann,, Erich Roland]","[January 20, 2013, (, 2013-01-20, ), (Sundance...",United States,English,...,Joe Bini,Philip Sheppard,"[Passion Pictures, ,, Motto Pictures,, John Ba...",NaN,NaN,tt2475544,NaN,NaN,2013-01-20,100
5621,https://en.wikipedia.org/wiki/Lawless_(film),2012,https://www.imdb.com/title/tt1212450/,Lawless,"[The Wettest County in the World, by, Matt Bon...","[Shia LaBeouf, Tom Hardy, Gary Oldman, Mia Was...",Benoît Delhomme,"[May 19, 2012, (, 2012-05-19, ), (, Cannes, ),...",United States,English,...,Dylan Tichenor,"[Nick Cave, Warren Ellis]","[Lucy Fisher, Douglas Wick, Megan Ellison, Mic...","[Red Wagon Entertainment, Annapurna Pictures, ...",Nick Cave,tt1212450,54400000.0,26000000.0,2012-05-19,115
1199,https://en.wikipedia.org/wiki/Coldblooded_(film),1995,https://www.imdb.com/title/tt0112702/,Coldblooded,NaN,"[Jason Priestley, Peter Riegert, Kimberly Will...",Robert D. Yeoman,"[September 15, 1995, (, 1995-09-15, ), (U.S.),...",United States,English,...,Craig Bassett,Steve Bartek,"[Larry Estes, Michael J. Fox, Brad Jenkel, Bra...","[PolyGram Filmed Entertainment, Propaganda Films]",Wallace Wolodarsky,tt0112702,16198.0,NaN,1995-09-15,92
939,https://en.wikipedia.org/wiki/Disclosure_(film),1994,https://www.imdb.com/title/tt0109635/,Disclosure,"[Disclosure, by Michael Crichton]","[Michael Douglas, Demi Moore, Donald Sutherlan...",Tony Pierce-Roberts,"[December 9, 1994, (, 1994-12-09, )]",United States,English,...,Stu Linder,Ennio Morricone,"[Michael Crichton, Barry Levinson]",NaN,Paul Attanasio,tt0109635,214000000.0,55000000.0,1994-12-09,128
1643,https://en.wikipedia.org/wiki/Mad_Dog_Time,1996,https://www.imdb.com/title/tt0116953/,Mad Dog Time,NaN,"[Ellen Barkin, Gabriel Byrne, Richard Dreyfuss...",Frank Byers,"[November 8, 1996, (, 1996-11-08, )]",United States,English,...,Norman Hollyn,Earl Rose,Judith James,"[Bruin Grip Services, Dreyfuss/James Productio...",Larry Bishop,tt0116953,107874.0,8000000.0,1996-11-08,93
274,https://en.wikipedia.org/wiki/Blue_Desert_(film),1991,https://www.imdb.com/title/tt0101486/,Blue Desert,NaN,"[Courteney Cox, D. B. Sweeney, Craig Sheffer, ...",Paul Murphy,"[January 9, 1991, (, 1991-01-09, )]",United States,English,...,Debra Bard,Joel Goldsmith,NaN,NaN,"[Arthur Collis, Bradley Battersby]",tt0101486,NaN,NaN,1991-01-09,98
5789,https://en.wikipedia.org/wiki/The_Butler,2013,https://www.imdb.com/title/tt1327773/,The Butler,"[A Butler Well Served By This Election, by, Wi...","[Forest Whitaker, Oprah Winfrey, Mariah Carey,...",Andrew Dunn,"[August 16, 2013, (, 2013-08-16, )]",United States,English,...,Joe Klotz,Rodrigo Leão,"[Pamela Oas Williams, Laura Ziskin, Lee Daniel...","[Laura Ziskin Productions, Windy Hill Pictures...",Danny Strong,tt1327773,176600000.0,30000000.0,2013-08-16,132
5671,https://en.wikipedia.org/wiki/Red_Lights_(2012...,2012,https://www.imdb.com/title/tt1748179/,Red Lights,NaN,"[Cillian Murphy, Sigourney Weaver, Elizabeth O...",Xavi Giménez,"[January 20, 2012, (, 2012-01-20, ), (Sundance...","[Spain, United States]",English,...,Rodrigo Cortés,Victor Reyes,"[Rodrigo Cortés, Adrián Guera]",NaN,Rodrigo Cortés,tt1748179,13600000.0,NaN,2012-01-20,113
3739,https://en.wikipedia.org/wiki/Everything_Is_Il...,2005,https://www.imdb.com/title/tt0404030/,Everything Is Illuminated,"[Everything Is Illuminated, by, Jonathan Safra...","[Elijah Wood, Eugene Hütz, Boris Leskin, Larys...",Matthew Libatique,"[September 16, 2005, (, 2005-09-16, )]",United States,"[English, Russian, Ukrainian]",...,"[Andrew Marcus, Craig McKay]","[Paul Cantelon, Sergei Shnurov]","[Marc Turtletaub, Peter Sa